In [1]:
# Import Tecton and other libraries
import logging
import os
import tecton
from dotenv import load_dotenv
import pandas as pd
import snowflake.connector
from datetime import datetime, timedelta
from pprint import pprint

load_dotenv()  # take environment variables from .env.
logging.getLogger('snowflake.connector').setLevel(logging.WARNING)
logging.getLogger('snowflake.snowpark').setLevel(logging.WARNING)

connection_parameters = {
    "user": INSERT_USER,
    "password": INSERT_PASSWROD,
    "account": "tectonpartner",
    "warehouse": "DEMO_WH",
    # Database and schema are required to create various temporary objects by tecton
    "database": "SANIKA",
    "schema": "PUBLIC",
}
conn = snowflake.connector.connect(**connection_parameters)
tecton.snowflake_context.set_connection(conn) # Tecton will use this Snowflake connection for all interactive queries


# Quick helper function to query snowflake from a notebook
# Make sure to replace with the appropriate connection details for your own account
def query_snowflake(query):
    df = conn.cursor().execute(query).fetch_pandas_all()
    return df

tecton.version.summary()

Version: 0.4.6
Git Commit: bf6de72abf3416be21916d94abcc671e4246d4d0
Build Datetime: 2022-07-19T23:22:05


In [27]:
ws = tecton.get_workspace('sanika-snowflake-live')

In [28]:
fv = ws.get_feature_view('user_transaction_metrics')

In [29]:
start_time = datetime.utcnow()-timedelta(days=120)
end_time = datetime.utcnow()

In [31]:
fv.run(start_time=start_time, end_time=end_time).to_pandas()

WARNING - 07/22/2022 01:31:19 PM - InteractiveRunApi - Run time range (2022-03-24 17:30:25.153839+00:00, 2022-07-22 17:30:25.153883+00:00) is not a multiple of the aggregation_interval: 1 day, 0:00:00. This may lead to incorrect aggregate feature values, since Tecton pre-aggregates data in smaller time windows based on the aggregation_interval size.


,USER_ID,TIMESTAMP,INT_VALUE_VAR_SAMP_1D_1D,INT_VALUE_STDDEV_SAMP_1D_1D,INT_VALUE_STDDEV_POP_1D_1D,INT_VALUE_VAR_POP_1D_1D,FLOAT_VALUE_VAR_SAMP_1D_1D,FLOAT_VALUE_STDDEV_SAMP_1D_1D,FLOAT_VALUE_STDDEV_POP_1D_1D,FLOAT_VALUE_VAR_POP_1D_1D
0,user_a,2022-05-15,1.200000e+19,3.464102e+09,3.000000e+09,9.000000e+18,1.200000e+19,3.464102e+09,3.000000e+09,9.000000e+18
1,user_b,2022-05-15,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00
2,user_c,2022-05-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,user_d,2022-05-15,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN
4,user_d,2022-05-16,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN
5,user_c,2022-05-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,user_b,2022-05-16,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00
7,user_b,2022-05-17,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00
8,user_a,2022-05-18,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN
9,user_a,2022-05-17,5.000000e-01,7.071068e-01,5.000000e-01,2.500000e-01,5.000000e-01,7.071068e-01,5.000000e-01,2.500000e-01


In [26]:
fv.get_historical_features(start_time=start_time, end_time=end_time).to_pandas().head()

,USER_ID,TIMESTAMP,INT_VALUE_VAR_SAMP_1D_1D,INT_VALUE_STDDEV_SAMP_1D_1D,INT_VALUE_STDDEV_POP_1D_1D,INT_VALUE_VAR_POP_1D_1D
0,user_a,2022-07-22,0.333333,0.57735,0.5,0.25
1,user_c,2022-07-22,NaN,NaN,NaN,NaN
2,user_d,2022-07-22,0.000000,0.00000,0.0,0.00


In [15]:
fs = ws.get_feature_service('fraud_detection_feature_service')

In [21]:
tecton.conf.set("TECTON_API_KEY", "<key>")

keys = {
    'USER_ID': 'user_a',
}
features = fs.get_online_features(join_keys=keys).to_dict()
pprint(features)

{'user_transaction_metrics.INT_VALUE_STDDEV_1D_1D': 0.0,
 'user_transaction_metrics.INT_VALUE_VARIANCE_1D_1D': 0.0}


curl -X POST --silent https://dev-snowflake.tecton.ai/api/v1/feature-service/get-features\
     -H "Authorization: Tecton-key $TECTON_API_KEY" -d\
'{
  "params": {
    "feature_service_name": "fraud_detection_feature_service",
    "join_key_map": {
      "USER_ID": "user_461615966685",
      "CATEGORY": "grocery_net"
    },
    "workspace_name": "prod"
  }
}' | jq